In [6]:
# external imports
import numpy as np
import tensorflow as tf

# internal imports
import data_generation as dg
from autoencoder import Autoencoder
from fixed_point import find_fixed_points
from visualize_model_dynamics import visualize

In [7]:
# data generation
n_points = 100
data_name = 'helix'
X = np.linspace(0, 10, n_points)
X = dg.FUNC_DICT[data_name](X)
np.random.shuffle(X)
train_ratio = 0.8
X_train, X_test = X[:int(train_ratio*n_points)], X[int(train_ratio*n_points):]

In [22]:
# create model
np.random.seed(42)
data_dim = 3
encoding_dim = 2
model = Autoencoder(input_dim=data_dim, encoding_dim=encoding_dim, hidden_layers=[])

# train model
epochs = 10
model.train(X_train, epochs=epochs, batch_size=32, path='models/'+data_name)

Epoch 1/10


1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step - accuracy: 0.5938 - loss: 147.2132
Epoch 1: saving model to models/helix/cp-0001.weights.h5
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7398 - loss: 135.3317  
Epoch 2/10
1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9688 - loss: 187.5986
Epoch 2: saving model to models/helix/cp-0002.weights.h5
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9453 - loss: 144.9732 
Epoch 3/10
1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9375 - loss: 95.0819
Epoch 3: saving model to models/helix/cp-0003.weights.h5
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9414 - loss: 88.9448 
Epoch 4/10
1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9688 - loss: 84.1521
Epoch 4: saving model to models/helix/cp-0004.weights.h5
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9414 - loss: 79.2268 
Epoch 5/10
1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9062 - loss: 70.5988
Epoch 5: saving model to models/helix/cp-0005.weights.h5
3/3 ━━━━

In [23]:
# load model
data_name = 'helix_100_points'
model_id = '0010'
model_path = f'models/{data_name}/cp-{model_id}.weights.h5'
model._autoencoder.load_weights(model_path)
model._autoencoder.summary()

# find fixed points
initial_guess = np.array([1,1,1])
stable_fixed_points, unstable_fixed_points = find_fixed_points(model)

Model: "autoencoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 3)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 2)              │             8 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 3)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19 (80.00 B)

 Trainable params: 17 (68.00 B)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

UnboundLocalError: cannot access local variable 'current_guess' where it is not associated with a value

In [ ]:
# create point grid
grid_width = 6
grid_intervals = np.array([[-10, 10],
                           [-10, 10],
                           [-10, 10]
])
grid = np.array(np.meshgrid(np.linspace(grid_intervals[0,0], grid_intervals[0,1], grid_width),
                            np.linspace(grid_intervals[1,0], grid_intervals[1,1], grid_width),
                            np.linspace(grid_intervals[2,0], grid_intervals[2,1], grid_width),
                            sparse=False))
grid = grid.T.reshape(-1, 3)

# study autoencoder operating on grid
reconstructed_grid = model.predict(grid)
ae_dynamics = reconstructed_grid - grid

# study autoencoder operating on test data
x_test_reconstructed = model.predict(X_test)
error = np.linalg.norm(x_test_reconstructed - X_test)
error = round(error,2)

# load fixed points
visualize(grid, ae_dynamics, error, stable_fixed_points, unstable_fixed_points)